In [ ]:
STATIC = False

In [ ]:
%matplotlib inline
import math as m
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
from ipywidgets import fixed, interactive_output
import ipywidgets as widgets

sns.set()
pd.options.display.max_columns = None
pd.options.display.max_rows = 5

In [ ]:
df = pd.read_csv("cyl_raw.csv")

In [ ]:
#CYCLE IDENTIFIZIEREN
# 180°: Ansaugen, 180°: Verdichten,...
d = {0:"Ansaugen",1:"Verdichten",2:"Arbeiten",3:"Ausstossen"}
cycle = [0]*180 + [1]*180 + [2]*180 + [3]*180
cycles = cycle*len(df["Drehzahl"].unique())
df["Cycle"]  = cycles

In [ ]:
#VOLUMEN
# Konstanten
RPMs = df["Drehzahl"].unique()
Vh = (73/2)**2 * m.pi * 76 #mm³
VH = 3*Vh
Hu = 41.636 #MJ/kg

def volume(winkel, h=76,l=135,dea=-0.6,s=0,eps=10,b=73):
    """
    params:
        winkel: Kurbelwinkel [°]
        h: Kolbenhub [mm]
        l: Pleuellänge [mm]
        dea: Deachsierung [mm]
        s: Schränkung [mm]
        eps: Verdichtungsverhältnis [1]
        b: Bohrung [mm]
    returns: 
        Volumen in Abh. von winkel [mm**3]
    """
    v_hub = (b/2)**2 * m.pi * h #mm**3
    x = h/2 * (1 - m.cos(m.radians(winkel))) + l * (1 - m.sqrt(1 - ((h*m.sin(m.radians(winkel)))/(2*l) - (dea-s)/l)**2))
    v = v_hub/(eps-1) + b**2 *(m.pi/4) * x
    return v

df["Volumen"] = df["Kurbelwinkel"].map(volume)

x = np.linspace(-360,360,720)
y = list(map(lambda w:volume(w),x))

fig = plt.figure(figsize=(6,3))
plt.xlabel("Kurbelwinkel [°]")
plt.ylabel("Volumen [mm³]")
plt.title("Kolbenbewegung Volumenverlauf")
plt.plot(x,y)
print(f"Hubvolumen: {volume(180)-volume(0)} mm³")
print(f"Hubram: {(volume(180)-volume(0))*3*10**(-3)} ccm")

In [ ]:
#DRUCK MITTELWERT
df["P_ZYLm"] = df.loc[:,["P_ZYL1","P_ZYL2","P_ZYL3"]].mean(axis=1)
#DRUCK GRADIENT
df["dp"] = df.loc[:,["Drehzahl","P_ZYLm"]].groupby(by="Drehzahl").diff()

In [ ]:
#DRUCKVERLAUF
fig, ax = plt.subplots(1,1,figsize=(11,4))
sns.lineplot(data=df,x="Kurbelwinkel",y="P_ZYLm",hue="Drehzahl",palette="tab10",ax=ax)
ax.set_title("Druckverlauf")
ax.set_xlabel("Kurbelwinkel [°]")
ax.set_ylabel("Druck [bar]")
plt.legend()
plt.show()

In [ ]:
#MAXIMALDRUCK

df_max = df.groupby(by="Drehzahl",as_index=False).max()

fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(7,3),sharey=True)

sns.barplot(data=df_max,x="Drehzahl",y="P_ZYLm",ax=axs[0])
axs[0].set_ylabel("Maximaldruck [bar]")
axs[0].set_title("Maximaldruck")
sns.regplot(data=df_max,x="Drehzahl",y="P_ZYLm",ax=axs[1],order=1)
axs[1].set_xlim(left=1200,right=4800)
axs[1].set_title("Regressionsgerade Maximaldruck-Drehzahl")
axs[1].set_ylabel("")
display(df_max.loc[:,["Drehzahl","P_ZYLm"]].rename(columns={"P_ZYLm":"P_ZYLmax"}))
plt.show()

In [ ]:
#DRUCKVERLAUF UND ZZP
def ZZP(rpm,log=False,ax=None,over="volume",vline=False,color="red"):
    '''
    params:
        rpm: Drehzahl in [1600,2100,2600,3600,4600]
        log: bool, True: y-Achse log skala
        ax: default: None, erstellt fig,ax. Sonst existiedendes plt.axes, um darauf zu plotten
        over: x-achse über der zu plotten ist in ["volume","angle"]
    '''
    df_rpm = df[df["Drehzahl"]==rpm]
    ZZP = df_rpm["ZZP"].unique()[0]
    
    
    if not ax:
        fig,ax = plt.subplots()
        
    if over == "volume":
        x = volume(ZZP)
    elif over == "angle":
        x = ZZP
    
    if vline:
        ax.axvline(x=x,color=color,label="ZZP")
    else:
        ax.scatter(x,df_rpm[(df_rpm["Cycle"]==1) & (df_rpm["Kurbelwinkel"]==ZZP)]["P_ZYLm"],color=color,label="ZZP")

def p_angle(rpm,log=False,gradient=False,ax=None):
    '''
    params:
        rpm: Drehzahl
        ax: existierendes plt.axes object. falls None wird eines erstellt
    '''
    df_rpm = df[df["Drehzahl"]==rpm]
    
    if not ax:
        fig, ax = plt.subplots()
    
    ax.plot(df_rpm["Kurbelwinkel"],df_rpm["P_ZYLm"],label="p")
    ax.set_xlabel("Kurbelwinkel [°]")
    ax.set_ylabel("Druck [bar]")
    ax.set_title(f"Druckverlauf und -gradient {rpm} U/min")
    
    if gradient:
        #durck gradient plotten
        ax.plot(df_rpm["Kurbelwinkel"],df_rpm["dp"]*10,linestyle="dotted",color="green",label="scaled dp [10bar/°KW]")
    
    if log:
        ax.set_yscale("log")
        ax.set_ylabel("log Druck [bar]")

        
rpm_dropd = widgets.Dropdown(options = RPMs,value=2100,description="Drehzahl")

def plot1(rpm,**kwargs):
    if not "ax" in kwargs.keys():
        fig, ax = plt.subplots(figsize=(9,4))
    else:
        ax = kwargs["ax"]
    
    p_angle(rpm,ax=ax,gradient=True)
    ZZP(rpm,ax=ax,over="angle")
    plt.legend()


if STATIC:
    fig = plt.figure(figsize=(12,12))
    gs = plt.GridSpec(3,2,figure=fig)
    plot1(1600,ax=fig.add_subplot(gs[0,:]))
    plot1(2100,ax=fig.add_subplot(gs[1,0]))
    plot1(2600,ax=fig.add_subplot(gs[1,1]))
    plot1(3600,ax=fig.add_subplot(gs[2,0]))
    plot1(4600,ax=fig.add_subplot(gs[2,1]))
    plt.tight_layout()
else:
    out1 = interactive_output(plot1,{"rpm":rpm_dropd})
    display(rpm_dropd,out1)

In [ ]:
#DRUCKGRADIENT UND ZZP
def plot_gradient(drehzahl,ax=None):
    df_rpm = df[df["Drehzahl"]==drehzahl]
    
    if not ax:
        fig,ax = plt.subplots(figsize=(9,3))
    
    
    ax.axvline(df_rpm["ZZP"].unique(),color="red",linewidth=0.75,label="ZZP")
    ax.plot(df_rpm["Kurbelwinkel"],df_rpm["dp"],label="dp")
    ax.set_title(f"Druckgradient {drehzahl} U/min")
    ax.set_xlabel("Kurbelwinkel [°]")
    ax.set_ylabel("[bar/1°KW]")
    
    ax.legend()
    
fig, axs =  plt.subplots(nrows=1,ncols=2,figsize=(10,3))
plot_gradient(1600,ax=axs[0])
plot_gradient(4600,ax=axs[1])

In [ ]:
#p-V DIAGRAMM
def pV(rpm,log=False,ax=None):
    '''
    Plotted ein p-V-Digramm für die angegebene Drehzahl {rpm}
    params:
        rpm:
        log:
        ax:
    '''
    d = {0:"Ansaugen",1:"Verdichten",2:"Arbeiten",3:"Ausstossen"}
    df_rpm = df[df["Drehzahl"]==rpm]
    
    if not ax:
        fig, ax = plt.subplots(1,1,figsize=(11,6))
    
    for cyc in df_rpm["Cycle"].unique():
        df_rpm_cyc = df_rpm[df_rpm["Cycle"]==cyc]
        ax.plot(df_rpm_cyc["Volumen"],df_rpm_cyc["P_ZYLm"],label=d[cyc])

    ax.set_xlabel("Volumen [mm³]")
    ax.set_ylabel("Druck [bar]")
    if log:
        ax.set_yscale("log")
        ax.set_ylabel("log Druck [bar]")
    
    plt.title(f"p-V Diagramm {rpm} U/min")
    
    
log_checkb = widgets.Checkbox(value=True,description="log Skala")

def plot2(rpm,log=None,**kwargs):
    if not "ax" in kwargs.keys():
        fig,ax = plt.subplots(figsize=(11,6))
    else:
        ax = kwargs["ax"]
        
    pV(rpm,log=log,ax=ax)
    ZZP(rpm,log=log,ax=ax)

    plt.legend()

if STATIC:
    log = True
    fig = plt.figure(figsize=(12,12))
    gs = plt.GridSpec(3,2,figure=fig)
    plot2(1600,log=log,ax=fig.add_subplot(gs[0,:]))
    plot2(2100,log=log,ax=fig.add_subplot(gs[1,0]))
    plot2(2600,log=log,ax=fig.add_subplot(gs[1,1]))
    plot2(3600,log=log,ax=fig.add_subplot(gs[2,0]))
    plot2(4600,log=log,ax=fig.add_subplot(gs[2,1]))
    plt.tight_layout()
else:
    out2 = interactive_output(plot2,{"rpm":rpm_dropd,"log":log_checkb})
    display(widgets.VBox([rpm_dropd,log_checkb]),out2)

In [ ]:
#ARBEIT HD LD SCHLEIFE
# HD -> cycle 2 - max(cycle1,cycle3)
# ND -> min(cycle3,cycle1) - cycle0
def HD(rpm,log=False,ax=None,plot=True):
    '''
    Berechnet die Hochdruckschleife als Kombination der Drucklinien der einzelnen Zyklen folgenderweise:
    HD nach oben durch Arbeits-Druckverlauf (Cycle=2) begrenzt und nach unten durch das Maximum von Ausstoß- (Cycle=3) 
    und Verdichtungs-verlauf (Cycle=1).
    ND nach unten durch Ansaug-verlauf (Cycle=0) begrenzt und nach oben durch das Minimum von Ausstoß (3) und
    Verdichtungs-verlauf (1).
    
    params:
        rpm:
        log:
        ax:
        plot: default True. False falls nur die HD Arbeit zurückgegeben werden soll.
    returns:
        HD Arbeit [bar*mm^3]
    '''
    df_rpm = df[df["Drehzahl"]==rpm]
    cyc2 = df_rpm[df_rpm["Cycle"]==2].loc[:,["Kurbelwinkel","Cycle","P_ZYLm","Volumen"]].sort_values(by="Kurbelwinkel")
    cyc1 = df_rpm.loc[(df_rpm["Cycle"]==1),["Kurbelwinkel","Cycle","P_ZYLm","Volumen"]].sort_values(by="Kurbelwinkel")
    cyc3 = df_rpm.loc[(df_rpm["Cycle"]==3),["Kurbelwinkel","Cycle","P_ZYLm","Volumen"]].sort_values(by="Kurbelwinkel")

    cyc1.rename(columns={col:col+"_1" for col in cyc1.columns},inplace=True)
    cyc3.rename(columns={col:col+"_3" for col in cyc3.columns},inplace=True)

    cyc13 = pd.concat([cyc1.reset_index(drop=True),cyc3.reset_index(drop=True)],axis=1)
    cyc13["P_ZYLm"] = cyc13[["P_ZYLm_1","P_ZYLm_3"]].max(axis=1)
    cyc13["Volumen"] = cyc13["Volumen_1"]

    HD = pd.DataFrame(data={
        "P_ZYLm":np.append(cyc13["P_ZYLm"].values,cyc2["P_ZYLm"].values),
        "Volumen":np.append(cyc13["Volumen"].values,cyc2["Volumen"].values)
    })
    
    if plot:
        if not ax:
            fig,ax = plt.subplots()
            
        ax.plot(HD["Volumen"],HD["P_ZYLm"],label="HD")
        
        ax.set_xlabel("Volumen [mm³]")
        ax.set_ylabel("Druck [bar]")
        if log:
            plt.yscale("log")
            ax.set_ylabel("log Druck [bar]")
    
        ax.fill_between(cyc2.sort_values(by="Volumen")["Volumen"],cyc2.sort_values(by="Volumen")["P_ZYLm"],
                        cyc13.sort_values(by="Volumen")["P_ZYLm"],label="W_HD",alpha=0.5)
    
    return np.trapz(HD["P_ZYLm"],x=HD["Volumen"]) #bar*mm^3

def ND(rpm,log=False,ax=None,plot=True):
    df_rpm = df[df["Drehzahl"]==rpm]
    cyc0 = df_rpm[df_rpm["Cycle"]==0].loc[:,["Kurbelwinkel","Cycle","P_ZYLm","Volumen"]].sort_values(by="Kurbelwinkel")
    cyc1 = df_rpm.loc[(df_rpm["Cycle"]==1),["Kurbelwinkel","Cycle","P_ZYLm","Volumen"]].sort_values(by="Kurbelwinkel")
    cyc3 = df_rpm.loc[(df_rpm["Cycle"]==3),["Kurbelwinkel","Cycle","P_ZYLm","Volumen"]].sort_values(by="Kurbelwinkel")

    cyc1.rename(columns={col:col+"_1" for col in cyc1.columns},inplace=True)
    cyc3.rename(columns={col:col+"_3" for col in cyc3.columns},inplace=True)

    cyc13 = pd.concat([cyc1.reset_index(drop=True),cyc3.reset_index(drop=True)],axis=1)
    cyc13["P_ZYLm"] = cyc13[["P_ZYLm_1","P_ZYLm_3"]].min(axis=1)
    cyc13["Volumen"] = cyc13["Volumen_1"]

    ND = pd.DataFrame(data={
        "P_ZYLm":np.append(cyc0["P_ZYLm"].values,cyc13["P_ZYLm"].values),
        "Volumen":np.append(cyc0["Volumen"].values,cyc13["Volumen"].values)
    })
    
    if plot:
        if not ax:
            fig,ax = plt.subplots()
            
        ax.plot(ND["Volumen"],ND["P_ZYLm"],label="ND")
        
        ax.set_xlabel("Volumen [mm³]")
        ax.set_ylabel("Druck [bar]")
        if log:
            plt.yscale("log")
            ax.set_ylabel("log Druck [bar]")
    
        ax.fill_between(cyc0.sort_values(by="Volumen")["Volumen"],cyc0.sort_values(by="Volumen")["P_ZYLm"],
                        cyc13.sort_values(by="Volumen")["P_ZYLm"],label="W_ND",alpha=0.5)
    
    return np.trapz(ND["P_ZYLm"],x=ND["Volumen"])
    
def plot3(rpm,log=None,**kwargs):
    if not "ax" in kwargs.keys():
        fig,ax = plt.subplots(figsize=(11,6))
    else:
        ax = kwargs["ax"]
        
    HD(rpm,log=log,ax=ax)
    ND(rpm,log=log,ax=ax)
    ZZP(rpm,log=log,ax=ax)
    plt.title(f"p-V Diagramm HD, ND bei {rpm} U/min")
    plt.legend()

rpm_dropd1 = widgets.Dropdown(options=RPMs,value=1600,description="Drehzahl")
log_checkb1 = widgets.Checkbox(value=True,description="log Skala")

if STATIC:
    log=True
    fig = plt.figure(figsize=(12,12))
    gs = plt.GridSpec(3,2,figure=fig)
    plot3(1600,log=log,ax=fig.add_subplot(gs[0,:]))
    plot3(2100,log=log,ax=fig.add_subplot(gs[1,0]))
    plot3(2600,log=log,ax=fig.add_subplot(gs[1,1]))
    plot3(3600,log=log,ax=fig.add_subplot(gs[2,0]))
    plot3(4600,log=log,ax=fig.add_subplot(gs[2,1]))
    plt.tight_layout()
else:
    out3 = interactive_output(plot3,{"rpm":rpm_dropd1,"log":log_checkb1})
    display(widgets.VBox([rpm_dropd1,log_checkb1]),out3)

In [ ]:
#INDIZIERTER MITTELDRUCK
pmiHD = [HD(rpm,plot=False)/Vh for rpm in RPMs] #bar, integgration der oberen Begrenzung in return value von HD()
pmiND = [ND(rpm,plot=False)/Vh for rpm in RPMs] #bar, interation der unteren Berenzung ND()
df_pm = pd.DataFrame({"Drehzahl":RPMs,"pmiHD":pmiHD,"pmiND":pmiND})
df_pm["pmi"] = df_pm["pmiHD"]+df_pm["pmiND"]
fig,axs = plt.subplots(1,3,figsize=(10,3),sharey=False)

df_pm.loc[:,["Drehzahl","pmiHD","pmiND"]].plot(kind="bar",x="Drehzahl",stacked=True,ax=axs[0])
axs[0].set_ylabel("Druck [bar]")
axs[0].set_title("Verhältnisse ND und HD an pmi")

df_pm.plot(kind="line",x="Drehzahl",ax=axs[1],style=["-o"]*3)
axs[1].set_xticks(RPMs)
axs[1].set_title("Zusammensetzung pmi aus ND und HD")

axs[2].plot(df_pm["Drehzahl"],abs(df_pm["pmiND"]/df_pm["pmi"]),"-o")
axs[2].set_xticks(RPMs)
axs[2].set_xlabel("Drehzahl")
axs[2].set_title("Betragsmäß. Anteil ND an pmi")

def style_negative(v, props=''):
    return props if v < 0 else None

display(df_pm.style.applymap(style_negative, props='color:red;'))
plt.tight_layout()
plt.show()

In [ ]:
#VERGLEICH TRAPEZINTEGRATION - KLASSISCH RIEMANN
df["dV"] = df.loc[:,"Volumen"].diff()
df["pmi_rs"] = df["dV"]*df["P_ZYLm"]/Vh
df_pmi = df.groupby(by="Drehzahl",as_index=False).sum().loc[:,["Drehzahl","pmi_rs"]]
df_pmi.rename(columns={"pmi":"pmi_rs"},inplace=True)
df_pmi["pmi_trpz"] = df_pm["pmi"]
df_pmi["delta"] = abs((df_pmi["pmi_trpz"]-df_pmi["pmi_rs"])/df_pmi["pmi_rs"])

fig, (ax1,ax2) = plt.subplots(1,2,figsize=(7,3))
df_pmi.iloc[:,:-1].plot(kind="bar",x="Drehzahl",ax=ax1)
plt.title("Relativer Fehler Riemann-Trapez")
ax1.set_ylabel("ind. Mitteldruck [bar]")
ax2.plot(df_pmi["Drehzahl"],df_pmi["delta"],"o")
ax2.set_xticks(RPMs)
plt.tight_layout()
plt.show()

In [ ]:
#EFFEKTIVER MITTELDRUCK
def pme(rpm,f=2):
    M_rpm = df[df["Drehzahl"]==rpm]["Drehmoment"].unique()[0] #Nm
    return 10*(2*f*m.pi*M_rpm*10**(3))/(VH) #bar | [M]=Nm; [VH]=mm^3

df_pm["pme"] = [pme(rpm) for rpm in RPMs]

In [ ]:
#REIBMITTELDRUCK
df_pm["pmr"] = df_pm["pmi"] - df_pm["pme"]

fig, ax = plt.subplots(figsize=(7,3))
df_pm.plot(kind="bar",x="Drehzahl",y="pmi",ax=ax,align="edge",width=-0.2,color="green")
df_pm.plot(kind="bar",x="Drehzahl",y=["pme","pmr"],stacked=True,ax=ax,align="edge",width=0.2)
ax.set_title("Zusammensetzung p_mi = p_me + p_mr")
ax.set_ylabel("[bar]")
#l,r = plt.xlim()
#plt.xlim(l, r+0.2)
display(df_pm.style.applymap(style_negative, props='color:red;'))

In [ ]:
#QB ZYKL
def QB_zykl(rpm,Hu=41.636):
    '''
    params:
        rpm: 1/min
        Hu [MJ/kg]
    returns:
        QB/Zykl [J]
    '''
    dm = df[df["Drehzahl"]==rpm]["Absolutverbrauch"].unique()[0] #kg/h
    return dm/60 * 10**6 * Hu * 1/rpm * 2 #kg/min * J/kg * min/1

df_zykl = pd.DataFrame(
    {"Drehzahl":RPMs,"Absolutverbrauch":df["Absolutverbrauch"].unique(),"QB_zykl":[QB_zykl(rpm) for rpm in RPMs]}
)
display(df_zykl)

fig, ax1 = plt.subplots(figsize=(9,4))
ax1.plot(df_zykl["Drehzahl"],df_zykl["QB_zykl"],"-o",label="QB_zykl",color="blue")
ax1.legend(loc=(0.5,0.9))
ax1.set_xticks(RPMs)
ax2 = ax1.twinx()
ax2.plot(df_zykl["Drehzahl"],df_zykl["Absolutverbrauch"],"-o",label="Absolutverbrauch",color="red")
ax2.legend(loc=(0.5,0.8))
plt.title("Absolutverbrauch und eingebrachte Treibstoffenergie pro Zyklus")
ax1.set_ylabel("QB_zykl [J]",color="blue")
ax1.tick_params("y",colors="blue")
ax2.set_ylabel("Absolutverbrauch [kg/h]",color="red")
ax2.tick_params("y",colors="red")


plt.show()

In [ ]:
#Wi ZYKL
df_zykl["Wi_zykl"] = 3*0.1*df_pm["pmi"]*Vh*10**(-3) #N/mm² * mm³ -> Nm == J

fig = plt.figure(figsize=(5,3))
plt.plot(df_zykl["Drehzahl"],df_zykl["Wi_zykl"],"-o")
plt.title("ind. Arbeit/Zykl")
plt.ylabel("Wi_zykl [J]")
plt.xlabel("Drehzahl")
plt.xticks(RPMs)

display(df_zykl)
plt.show()

In [ ]:
#We ZYKL
df_zykl["We_zykl"] = 3*0.1*df_pm["pme"]*Vh*10**(-3) #N/mm² * mm³ -> Nm == J
display(df_zykl)

fig, ax = plt.subplots(figsize=(5,3))
df_zykl.plot(x="Drehzahl",y=["QB_zykl","Wi_zykl","We_zykl"],style="-o",ax = ax)
ax.set_xticks(RPMs)
ax.set_ylabel("[J]")
ax.set_title("Zyklische Größen Q_B, Wi, We")
plt.show()

In [ ]:
#TEMPERATURVERLAUF
#Masse berechnen
def mass(rpm):
    df_rpm = df[df["Drehzahl"]==rpm]
    p = df_rpm[df_rpm["Kurbelwinkel"]==-120].loc[:,"P_ZYLm"].values[0] #bar
    V = df_rpm[df_rpm["Kurbelwinkel"]==-120].loc[:,"Volumen"].values[0] #mm³
    m = p*0.1 * V * 10**(-3) / (287*380) #(N/mm² * mm³)*10**(-3) -> Nm / J/kgK * K -> J/kg --> kg
    return m #kg
for rpm in RPMs:
    df.loc[df["Drehzahl"]==rpm,"m"] = mass(rpm)
    
#Temperatur berechnen
df["T"] = (df["P_ZYLm"]*0.1*df["Volumen"]*10**(-3))/(df["m"]*287) #K

#cv berechnen
df["cv"] = 0.7+0.255*df["T"]*10**(-3)

#kappa berechnen
df["kappa"] = 1+(0.2888/df["cv"])

#p2' berechnen: shifting p1 and v1 for one row p' = p_shift(v_shift/v)**kappa
for rpm in RPMs:
    df.loc[df["Drehzahl"]==rpm,"P_ZYLm_shift"] = df.loc[df["Drehzahl"]==rpm,"P_ZYLm"].shift(1)
    df.loc[df["Drehzahl"]==rpm,"Volumen_shift"] = df.loc[df["Drehzahl"]==rpm,"Volumen"].shift(1)

df["p'"] = df["P_ZYLm_shift"] * (df["Volumen_shift"]/df["Volumen"])**df["kappa"] #bar

#dQH berechnen: dQH = cv * V * (p-p')/R
df["dQH"] = df["cv"]*1000 * df["Volumen"] * 10**(-9) * (df["P_ZYLm"]-df["p'"]) * 10**(5) / 287

#Temperaturverlauf plotten
fig, ax = plt.subplots(figsize=(12,5))

sns.lineplot(data=df,x="Kurbelwinkel",y="T",hue="Drehzahl",palette="tab10",ax=ax)
ax.set_title("Temperaturverlauf")
ax.set_ylabel("Temperatur [K]")

plt.show()

In [ ]:
#GEMESSENER HEIZVERLAUF
fig, ax = plt.subplots(figsize=(12,5))
sns.lineplot(data=df,x="Kurbelwinkel",y="dQH",hue="Drehzahl",palette="tab10")
ax.set_title("Heizverlauf")
ax.set_ylabel("dQH [J/°KW]")
plt.show()

In [ ]:
#IDEALISIERTER HEIZVERLAUF
df.loc[:,"dQH_ideal"] = df["dQH"]

mask = (df["Kurbelwinkel"] < df["ZZP"]) | ((df["Kurbelwinkel"]> df["ZZP"]) & (df["dQH"] < 0))
df.loc[mask,"dQH_ideal"] = 0

fig,ax = plt.subplots(figsize=(12,5))
df[df["Drehzahl"]==2100].plot(x="Kurbelwinkel",y=["dQH","dQH_ideal"],ax=ax)
ZZP(2100,over="angle",ax=ax,vline=True,color="black")
ax.set_ylabel("dQH [J/°KW]")
ax.set_title(f"Gemessener und idealisierter Heizverlauf bei {2100} U/min")
plt.legend()
plt.show()

In [ ]:
#GEMESSENER INTEGRIERTER HEIZVERLAUF
for rpm in RPMs:
    df.loc[df["Drehzahl"]==rpm,"QH"] = df.loc[df["Drehzahl"]==rpm,"dQH"].expanding(1).sum()

fig, ax = plt.subplots(figsize=(12,5))
sns.lineplot(data=df,x="Kurbelwinkel",y="QH",hue="Drehzahl",palette="tab10")
ax.set_title("Gemessener Integrierter Heizverlauf")
ax.set_ylabel("QH [J]")

plt.show()

In [ ]:
#IDEALISIERTER INTEGRIERTER HEIZVERLAUF
for rpm in RPMs:
    df.loc[df["Drehzahl"]==rpm,"QH_ideal"] = df.loc[df["Drehzahl"]==rpm,"dQH_ideal"].expanding(1).sum()
    
fig,ax = plt.subplots(figsize=(12,5))
df[df["Drehzahl"]==2100].plot(x="Kurbelwinkel",y=["QH","QH_ideal"],ax=ax)
ZZP(2100,over="angle",ax=ax,vline=True,color="black")
ax.set_ylabel("QH [J]")
ax.set_title(f"Gemessener und idealisierter integrierter Heizverlauf bei {2100} U/min")
plt.legend()
plt.show()

In [ ]:
#Qw ZYKL
#als maximalwert
df_zykl["QH_max"] = df.groupby(by="Drehzahl",as_index=False).max().loc[:,"QH"]

#oder als endwert
#df_zykl["QH_max"] = df.groupby(by="Drehzahl",as_index=False).agg(lambda x: x.iloc[-1]).loc[:,"QH"]

df_zykl["QH_max_ideal"] = df.groupby(by="Drehzahl",as_index=False).max().loc[:,"QH_ideal"]

df_zykl["QH_zykl"] = df_zykl["QH_max"]*3
df_zykl["QH_zykl_ideal"] = df_zykl["QH_max_ideal"]*3

df_zykl["QW_zykl"] = df_zykl["QB_zykl"] - df_zykl["QH_zykl"]
df_zykl["QW_zykl_ideal"] = df_zykl["QB_zykl"] - df_zykl["QH_zykl_ideal"]

def verlustarbeit(ideal=False, ax=None):
    if not ax:
        fig,ax = plt.subplots(figsize=(7,3))
    attr = "_ideal" if ideal else ""
    bars1 = df_zykl.plot(kind="bar",x="Drehzahl",y=[f"QH_zykl{attr}",f"QW_zykl{attr}"],stacked=True,ax=ax,align="edge",
                         width=0.25)
    bars2 = df_zykl.plot(kind="bar",x="Drehzahl",y="QB_zykl",ax=ax,width=-0.25,align="edge",color="green")
    ax.set_xlim(left=ax.get_xlim()[0],right=ax.get_xlim()[1]+0.25)
    ax.set_title(f"QB, QH{attr} und QW{attr}")
    ax.set_ylabel("Q [J]")
    plt.show()

display(df_zykl.loc[:,["Drehzahl","QB_zykl","Wi_zykl","We_zykl","QH_zykl_ideal","QW_zykl_ideal"]])

verlustarbeit(ideal=False)
verlustarbeit(ideal=True)

In [ ]:
#DRUCK HEIZ INTEGRIERTER HEIZVERLAUF - IDEAL VS REAL
def verlauf(rpm,xlim=(-360,360),ideal=False,ax=None,umsatz=False):
    '''
    Plottet Druck und Heizverläufe
    params:
        rpm:
        xlim:
        ideal: bool: True für ideale Heizverläufe; False für aus Messung berechnete.
        ax:
        verbr: bool: True für Einfügen von 5, 50, 95 Umsatzpunkt
    '''
    df_rpm = df[df["Drehzahl"]==rpm]
    if not ax:
        fig,ax = plt.subplots(figsize=(12,6))
        
    x = df_rpm["Kurbelwinkel"]
    ax.plot(x,df_rpm["P_ZYLm"],label="Druckverlauf",color="blue")
    ax.tick_params(axis='y', colors='blue')
    ZZP(rpm,ax=ax,over="angle",color="black")
    ax.legend(loc=(0.05,0.8))
    ax.set_ylabel("Druckverlauf [bar]")
    ax.set_xlabel("Kurbelwinkel [°]")
    
    attr = "idealisierter" if ideal else "gemessener"
    
    ax.set_title(f"{attr} Druck- und Heizverlauf bei {rpm} U/min")
    
    if ideal:
        y1 = df_rpm["dQH_ideal"]
        y2 = df_rpm["QH_ideal"]
    else:
        y1 = df_rpm["dQH"]
        y2 = df_rpm["QH"]
    
    ax1 = ax.twinx()
    ax1.plot(x,y1*10,label="dQH skaliert",color="red") #skaliert mit faktor 10
    ax1.tick_params(axis='y', colors='red')
    ax1.plot(x,y2,label="QH",color="green")
    ax1.tick_params(axis='y', colors='green')
    ax1.set_ylabel("QH [J], dQH skaliert [0.1J/°KW]")
    ax1.grid(False)
    
    if umsatz:
        for pkt in [5,50,95]:
            x,y = umsatzpunkt(rpm,pkt,ideal=ideal)
            ax1.scatter(x,y,label=f"{pkt}% Punkt",marker="o")
    ax1.legend(loc=(0.7,0.5))   
    
    #align zero line
    ax_ylims = ax.axes.get_ylim()           # Find y-axis limits set by the plotter
    ax_yratio = ax_ylims[0] / ax_ylims[1]  # Calculate ratio of lowest limit to highest limit

    ax1_ylims = ax1.axes.get_ylim()           # Find y-axis limits set by the plotter
    ax1_yratio = ax1_ylims[0] / ax1_ylims[1]
    
    if ax_yratio < ax1_yratio: 
        ax1.set_ylim(bottom = ax1_ylims[1]*ax_yratio)
    else:
        ax.set_ylim(bottom = ax_ylims[1]*ax1_yratio)
        
    #kurbelwinkel range
    plt.xlim(left=xlim[0],right=xlim[1])
    
    return
    
rpm_dropd2 = widgets.Dropdown(options=RPMs,value=2600)
xlim_slider = widgets.IntRangeSlider(value=[-50,100],min=-360,max=360,continuous_update=False,description="Kurbelwinkel")

if STATIC:
    log=xlim=(-50,70)
    fig = plt.figure(figsize=(12,12))
    gs = plt.GridSpec(3,2,figure=fig)
    verlauf(1600,xlim=xlim,ax=fig.add_subplot(gs[0,:]))
    verlauf(2100,xlim=xlim,ax=fig.add_subplot(gs[1,0]))
    verlauf(2600,xlim=xlim,ax=fig.add_subplot(gs[1,1]))
    verlauf(3600,xlim=xlim,ax=fig.add_subplot(gs[2,0]))
    verlauf(4600,xlim=xlim,ax=fig.add_subplot(gs[2,1]))
    plt.tight_layout()
else:
    out4 = interactive_output(verlauf,{"rpm":rpm_dropd2,"xlim":xlim_slider,"ideal":fixed(False)})
    display(rpm_dropd2,xlim_slider,out4) 

In [ ]:
#UMSATZPUNKTE
def umsatzpunkt(rpm,pkt,ideal=False):
    '''
    params:
        rpm: Drehzhl
        pkt: Umsatzpunkt in %, e.g. one of int 5, 50, 95
    returns:
        Koordinaten des Umsatzpunktes (x,y) x: Kurbelwinkel [°KW], y: QH [J]
    '''
    
    if ideal:
        QH_max = df_zykl.loc[df_zykl["Drehzahl"]==rpm,"QH_max_ideal"].values[0]
        upkt = df.loc[(df["Drehzahl"]==rpm) & (df["QH_ideal"] >= 0.01*pkt*QH_max),["Kurbelwinkel","QH_ideal"]].iloc[0,:]

    else:
        QH_max = df_zykl.loc[df_zykl["Drehzahl"]==rpm,"QH_max"].values[0]
        upkt = df.loc[(df["Drehzahl"]==rpm) & (df["QH"] >= 0.01*pkt*QH_max),["Kurbelwinkel","QH"]].iloc[0,:]
    
    return tuple(upkt)

#Umsatzpunkte °KW in Tabelle schreiben
for upkt in [5,50,95]:
    df_zykl[f"UP{upkt}"] = [umsatzpunkt(rpm,upkt,ideal=False)[0] for rpm in RPMs]
    df_zykl[f"UP{upkt}_ideal"] = [umsatzpunkt(rpm,upkt,ideal=True)[0] for rpm in RPMs]

# ZZP in Tabelle sschreiben
df_zykl["ZZP"] = df.groupby(by="Drehzahl",as_index=False).mean().loc[:,"ZZP"]

#Zündverzug ZVZ
df_zykl["ZVZ"] = df_zykl["UP5"] - df_zykl["ZZP"]
df_zykl["ZVZ_ideal"] = df_zykl["UP5_ideal"] - df_zykl["ZZP"]

#Brenndauer BD
df_zykl["BD"] = df_zykl["UP95"] - df_zykl["UP5"]
df_zykl["BD_ideal"] = df_zykl["UP95_ideal"] - df_zykl["UP5_ideal"]


display(df_zykl)

In [ ]:
#VERGLEICH IDEALE - REALE KURVEN
ideal_radio = widgets.RadioButtons(options=[("Ideal",True),("Gemessen",False)])

if STATIC:
    xlim = (-30,50)
    ideal = True
    umsatz = True
    fig = plt.figure(figsize=(12,12))
    gs = plt.GridSpec(3,2,figure=fig)
    verlauf(1600,xlim=xlim,ideal=ideal,umsatz=umsatz,ax=fig.add_subplot(gs[0,:]))
    verlauf(2100,xlim=xlim,ideal=ideal,umsatz=umsatz,ax=fig.add_subplot(gs[1,0]))
    verlauf(2600,xlim=xlim,ideal=ideal,umsatz=umsatz,ax=fig.add_subplot(gs[1,1]))
    verlauf(3600,xlim=xlim,ideal=ideal,umsatz=umsatz,ax=fig.add_subplot(gs[2,0]))
    verlauf(4600,xlim=xlim,ideal=ideal,umsatz=umsatz,ax=fig.add_subplot(gs[2,1]))
    plt.tight_layout()
else:
    out5 = interactive_output(verlauf,{"rpm":rpm_dropd,"ideal":ideal_radio,"umsatz":fixed(True),"xlim":fixed((-30,50))})
    display(rpm_dropd, ideal_radio, out5)

In [ ]:
#BRENNDAUER
fig, ax = plt.subplots(figsize=(12,5))

for h,rpm in enumerate(RPMs):
    for i,attr in enumerate(["","_ideal"]):
        start, end = df_zykl[df_zykl["Drehzahl"]==rpm][f"UP5{attr}"], df_zykl[df_zykl["Drehzahl"]==rpm][f"UP95{attr}"]
        sp =  df_zykl[df_zykl["Drehzahl"]==rpm][f"UP50{attr}"]
        x = [start, sp, end]
        offset = 0.15 if i%2 == 0 else -0.15
        style= "dotted"  if i%2==0 else "solid"
        y = [h+offset]*len(x)
        ax.plot(x,y,linewidth=2,color=sns.color_palette()[h],linestyle=style,marker="|",markersize=10,markeredgewidth=1.5)
    
ax.set_xlim(-310,50)
ax.set_ylim(-1,5)
ax.set_yticks(list(range(len(RPMs))))
ax.set_yticklabels(RPMs)
ax.set_title("Ideale (unten) vs. gemessene Brenndauer (oben) je Drehzahl")
ax.set_xlabel("Kurbelwinkel [°KW]")
ax.set_ylabel("Drehzahl")

plt.legend(("Messung","Idealisiert"))

plt.show()

In [ ]:
#PLOT IDEALE BRENNDAUER
fig,ax = plt.subplots(figsize=(9,4))

for h,rpm in enumerate(RPMs):
    start, end = df_zykl[df_zykl["Drehzahl"]==rpm]["UP5_ideal"], df_zykl[df_zykl["Drehzahl"]==rpm]["UP95_ideal"]
    sp =  df_zykl[df_zykl["Drehzahl"]==rpm]["UP50_ideal"]
    zzp = df_zykl[df_zykl["Drehzahl"]==rpm]["ZZP"]
    x = [start, sp, end]
    y = [h]*len(x)
    BD_line = ax.plot(x,y,linewidth=2,color=sns.color_palette()[h],linestyle="solid",marker="|",
                      markersize=10,markeredgewidth=1.5,label="Brenndauer")
    ZZP_scatter = ax.scatter(zzp,h,color="black",label="ZZP")
    if rpm == 4600:
        #annotate
        plt.annotate("5",(start.values[0],y[0]))
        plt.annotate("50",(sp.values[0],y[0]))
        plt.annotate("95",(end.values[0],y[0]))
        
ax.set_xlim(-30,30)
ax.set_ylim(-1,5)
ax.set_yticks(list(range(len(RPMs))))
ax.set_yticklabels(RPMs)
ax.set_title("(Idealisierte) Brenndauer und ZZP je Drehzahl")
ax.set_xlabel("Kurbelwinkel [°KW]")
ax.set_ylabel("Drehzahl")

legend_line = mlines.Line2D([], [], color='grey', marker='|',markersize=10, label='Brenndauer')

plt.legend((legend_line,ZZP_scatter),("Brenndauer","ZZP"))
plt.show()

In [ ]:
#PLOT ZVZ BD UMSATZPUNKTE
fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(10,4))
df_zykl.plot(kind="line",x="Drehzahl",y=["ZVZ_ideal","BD_ideal"],style="-o",ax=axs[0])
axs[0].set_xticks(RPMs)
axs[0].set_ylabel("°KW")
axs[0].set_title("Zündverzug und Brenndauer")

df_zykl.plot(kind="line",x="Drehzahl",y=["UP5_ideal","UP50_ideal","UP95_ideal"],style="-o",ax=axs[1])
axs[1].set_xticks(RPMs)
axs[1].set_ylabel("°KW")
axs[1].set_title("Lage der 5%-, 50%- und 95%-Umsatzpunkte")
axs[1].legend(loc=(0.6,0.5))

plt.show()

In [ ]:
#PLOT ZVZ ZZP
fig, ax = plt.subplots(figsize=(9,3))

df_zykl.plot(x="Drehzahl",y="ZVZ_ideal",ax=ax,marker="o",color="blue")
ax1 = ax.twinx()
df_zykl.plot(x="Drehzahl",y="ZZP",ax=ax1,linestyle="--",marker="x",color="red")
ax.legend(loc=(0.8,0.5))
ax1.legend(loc=(0.8,0.4))
ax.set_xticks(RPMs)
ax.tick_params("y",colors="blue")
ax.set_ylabel("ZVZ [°KW]")
ax1.set_ylabel("ZZP [°KW]")
ax1.tick_params("y",colors="red")
ax1.grid(False)

plt.show()

In [ ]:
#WIRKUNGSGRADE
eta_th = 1- (1/10**(1.4-1))

#druck daten und zykl. daten in eine tabelle nach drehzahl zusammenführen
df_rpm = pd.merge(df_zykl,df_pm,on="Drehzahl")

df_rpm["eta_th"] = eta_th
df_rpm["eta_i"] = df_rpm["Wi_zykl"]/df_rpm["QB_zykl"]
df_rpm["eta_g"] = df_rpm["eta_i"]/df_rpm["eta_th"]
df_rpm["eta_m"] = df_rpm["pme"]/df_rpm["pmi"]
df_rpm["eta_e"] = df_rpm["We_zykl"]/df_rpm["QB_zykl"]

def plot_eta(ax=None):
    if not ax:
        fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(11,4),sharey=True)
    
    df_rpm.plot(kind="bar",x="Drehzahl",y=["eta_th","eta_i","eta_e"],ax=axs[0])
    df_rpm.plot(kind="line",x="Drehzahl",y=["eta_th","eta_i","eta_e"],ax=axs[1],style="o-")
    axs[1].set_xticks(RPMs)
display(df_rpm)
plot_eta()

In [ ]:
#PLOT GESAMTWIRKUNGSGRAD
plt.plot(df_rpm["Drehzahl"],df_rpm["eta_e"],"-o")
plt.xticks(RPMs)
plt.xlabel("Drehzahl")
plt.title("Gesamtwirkungsgrad")
plt.show()

In [ ]:
#PLOOT GUETEGRAD ETA_M
fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(9,4),sharey=True)
df_rpm.plot(x="Drehzahl",y="eta_g",ax=axs[0],style="o-")
df_rpm.plot(x="Drehzahl",y="eta_m",ax=axs[1],style="o-")
axs[0].set_xticks(RPMs)
axs[0].set_title("Gütegrad")
axs[1].set_xticks(RPMs)
axs[1].set_title("Mechanischer Wirkunsgrad")

plt.show()

In [ ]:
#SPEZIFISCHER VERBRAUCH
#Table mit input/Output (verbrauch/ausstoß)
io = ['Drehzahl','Drehmoment','Absolutverbrauch', 'Angesaugte Luftmasse', 'O2 roh', 'CO roh',
       'CO2 roh', 'NOx roh', 'HC3 roh', 'Lambda', 'Temp Kühlm.E',
       'Temp Kühlm.A', 'Temp Oel', 'Temp Ansaugl. nach Verd.',
       'Temp Ansaugluft Saugrohr', 'Temp Abgas vor Turbine', 'P Saugrohr',
       'P Abg. vor Turb.']
df_io = df.loc[:,io].drop_duplicates().reset_index(drop=True)

# Ergebnistabelle mit allen Drehzahlvariablen Berechnungen
df_final = pd.merge(df_rpm,df_io.drop(columns=["Absolutverbrauch"]),on="Drehzahl",how="outer")

#spezifischer Verbrauch
df_final["be"] = 3600/(Hu * df_final["eta_e"])

fig, ax = plt.subplots(figsize=(8,3))
ax.plot(df_final["Drehzahl"],df_final["be"],"-o",label="spez. Verbrauch")
ax.set_xticks(RPMs)
ax.set_title("spezif. Kraftstoffverbrauch")
ax.set_ylabel("g/kWh")
ax.set_xlabel("Drehzahl U/min")

display(df_final)
plt.show()

In [ ]:
#PLOT ANSAUGUNG
fig,axs = plt.subplots(nrows=1,ncols=2,figsize=(9,4),gridspec_kw={'width_ratios': [3, 2]})

df_final.plot(kind="bar",x="Drehzahl",y=["Temp Ansaugluft Saugrohr","Temp Ansaugl. nach Verd."],ax=axs[0],stacked=False)

ax01 = axs[0].twinx()
df_final.loc[:,"P Saugrohr"].plot(kind="line",color="green",label="P Saugrohr",ax=ax01)
ax01.set_ylabel("mbar",color="green")
ax01.tick_params("y",labelcolor="green")
ax01.grid(False)

axs[1].plot(df_final["Drehzahl"],df_final["Angesaugte Luftmasse"],"-o")

axs[0].set_title("Ansaugluft-Temperatur und Saugrohrdruck")
axs[0].set_ylabel("°C")
axs[0].legend()

axs[1].set_title("Angesaugte Luftmasse")
axs[1].set_xticks(RPMs)
axs[1].set_ylabel("kg/h")
ax01.legend()

plt.tight_layout()
plt.show()

In [ ]:
#PLOT PARTIKEL EMISSIONEN
fig, ax = plt.subplots()
ax.plot(df_final["Drehzahl"],df_final["NOx roh"],"-o",label="NOx")
ax.plot(df_final["Drehzahl"],df_final["HC3 roh"],"-o",label="HC3")
ax.set_ylabel("ppm")
ax.set_title("Partikelemissionen")
ax.set_xticks(RPMs)
ax.legend()
plt.show()

In [ ]:
#PLOT VOL EMISSIONEN
fig, ax = plt.subplots()
ax.plot(df_final["Drehzahl"],df_final["O2 roh"],"-o",label="O2")
ax.plot(df_final["Drehzahl"],df_final["CO roh"],"-o",label="CO")
ax1 = ax.twinx()
ax1.plot(df_final["Drehzahl"],df_final["CO2 roh"],"-o",label="CO2",color="green")
ax1.grid(False)
ax1.legend(loc="lower right")
ax1.tick_params("y",color="green",labelcolor="green")
ax1.set_ylabel("Vol% CO2",color="green")

ax.set_ylabel("Vol%")
ax.set_xticks(RPMs)
ax.set_title("Vol% Emissionen")
ax.legend()
plt.show()

In [ ]:
#REGRESSIONSANALYSE
# inf color generator
def color_gen(colors = sns.color_palette("tab10")):
    i = 0
    while True:
        yield colors[i%len(colors)]
        i += 1
color_generator = color_gen()

params=["Drehzahl","be","ZZP","ZVZ_ideal","BD_ideal","UP50_ideal",'Angesaugte Luftmasse','O2 roh', 'CO roh',
       'CO2 roh', 'NOx roh', 'HC3 roh','eta_e','pmi',"pmiND","pmiHD"]


fig, axs = plt.subplots(nrows=5, ncols =3, figsize=(12,12))

i = 1
for row in range(5):
    for col in range(3):
        sns.regplot(data=df_final,x="Drehzahl",y=params[i],ax=axs[row,col],color=next(color_generator))
        axs[row,col].set_xticks(RPMs)
        axs[row,col].set_title(params[i])
        axs[row,col].set_ylabel("")
        i += 1
plt.tight_layout()
plt.show()

In [ ]:
#HEATMAP
#heatmap df_final aller Wirkungsgrade
#heatmap df_final aller io

from sklearn.preprocessing import minmax_scale
#scaling eta spaltenweise für heatmap
df_eta = df_final.set_index("Drehzahl").loc[:,["eta_i","eta_g","eta_m","eta_e"]]
scaled_eta = minmax_scale(df_eta)
df_scaled_eta = pd.DataFrame(data=scaled_eta,columns=["eta_i","eta_g","eta_m","eta_e"],index=RPMs)

#scaling IOs spaltenweise für heatmap
df_io = df_final.set_index("Drehzahl").loc[:,["be","O2 roh","CO roh","CO2 roh","NOx roh","HC3 roh"]]
scaled_io = minmax_scale(df_io)
df_scaled_io = pd.DataFrame(data=scaled_io,columns=["be","O2 roh","CO roh","CO2 roh","NOx roh","HC3 roh"],index=RPMs)

fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(9,4))
sns.heatmap(df_scaled_eta,linewidths=0.25,annot=df_eta,fmt='.3g',cmap="RdYlGn",cbar=False,ax=axs[0])
sns.heatmap(-df_scaled_io,linewidths=0.25,annot=df_scaled_io,cmap="RdYlGn",cbar=False,ax=axs[1])

axs[0].set_title("Wirkungsgrade")
axs[1].set_title("Verbrauch & Emissionen - skaliert (0,1)")

display(df_eta,df_io)
plt.show()

In [ ]:
#EXPORT ERGEBNISTABELLE IN XLSX
#with pd.ExcelWriter("master_table.xlsx") as writer:
#    for rpm in RPMs:
#        df[df["Drehzahl"]==rpm].to_excel(writer, sheet_name=str(rpm),index=False)
#    df_final.to_excel(writer,sheet_name="final",index=False)